# Evolució al llarg del temps

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math 

# Variables 

dataframes_naix = ['data/2024_pad_mdbas_lloc-naix-pais_lloc-naix-continent_sexe.csv',
'data/2023_pad_mdbas_lloc-naix-pais_lloc-naix-continent_sexe.csv',
'data/2022_pad_mdbas_lloc-naix-pais_lloc-naix-continent_sexe.csv',
'data/2021_pad_mdbas_lloc-naix-pais_lloc-naix-continent_sexe.csv',
'data/2020_pad_mdbas_lloc-naix-pais_lloc-naix-continent_sexe.csv',
'data/2019_pad_mdbas_lloc-naix-pais_lloc-naix-continent_sexe.csv',
'data/2018_pad_mdbas_lloc-naix-pais_lloc-naix-continent_sexe.csv',
'data/2017_pad_mdbas_lloc-naix-pais_lloc-naix-continent_sexe.csv',
'data/2016_pad_mdbas_lloc-naix-pais_lloc-naix-continent_sexe.csv',
'data/2015_pad_mdbas_lloc-naix-pais_lloc-naix-continent_sexe.csv',
'data/2014_pad_mdbas_lloc-naix-pais_lloc-naix-continent_sexe.csv',
'data/2013_pad_mdbas_lloc-naix-pais_lloc-naix-continent_sexe.csv',
'data/2012_pad_mdbas_lloc-naix-pais_lloc-naix-continent_sexe.csv',
'data/2011_pad_mdbas_lloc-naix-pais_lloc-naix-continent_sexe.csv',
'data/2010_pad_mdbas_lloc-naix-pais_lloc-naix-continent_sexe.csv',
'data/2009_pad_mdbas_lloc-naix-pais_lloc-naix-continent_sexe.csv',
'data/2008_pad_mdbas_lloc-naix-pais_lloc-naix-continent_sexe.csv',
'data/2007_pad_mdbas_lloc-naix-pais_lloc-naix-continent_sexe.csv',
'data/2006_pad_mdbas_lloc-naix-pais_lloc-naix-continent_sexe.csv',
'data/2005_pad_mdbas_lloc-naix-pais_lloc-naix-continent_sexe.csv',
'data/2004_pad_mdbas_lloc-naix-pais_lloc-naix-continent_sexe.csv',
'data/2003_pad_mdbas_lloc-naix-pais_lloc-naix-continent_sexe.csv',
'data/2002_pad_mdbas_lloc-naix-pais_lloc-naix-continent_sexe.csv']

# Funcions

## Processament

def calcul_pesos():
    gdf = gpd.read_file("data/seccio-censal.geojson")

    neighbours_named = {}

    for index, area in gdf.iterrows():
        name = index
        touching_names = gdf[gdf.geometry.touches(area.geometry)].index.tolist()
        neighbours_named[name] = touching_names
        
    w = [[0]*len(gdf) for _ in range (len(gdf))]

    for i,neighbours in neighbours_named.items():
        for j in neighbours:
            w[i][j] = 1

    return np.array(w)

## Càlculs

def calcul_morans_1(dataframe_name, pais, w):

    dataframe =  pd.read_csv(dataframe_name)
    dataframe['Valor'] = pd.to_numeric(dataframe['Valor'].replace('..', 2))

    seccions_unique = set(dataframe['Seccio_Censal'])

    if pais == 58:
        immigrants_df = dataframe[dataframe['LLOC_NAIX_PAIS'] != pais]
    else:
        immigrants_df = dataframe[dataframe['LLOC_NAIX_PAIS'] == pais]

    seccions_mancants = [seccio for seccio in seccions_unique if not (seccio in list(immigrants_df['Seccio_Censal']))]
    seccions_mancants_df = pd.DataFrame({'Seccio_Censal': seccions_mancants, 'Valor': [0]*len(seccions_mancants)})
    immigrants_df = pd.concat([immigrants_df, seccions_mancants_df], ignore_index=True)

    immigrants_df['sumats'] = immigrants_df.groupby('Seccio_Censal')['Valor'].transform('sum')
    immigrants_df = immigrants_df.drop_duplicates('Seccio_Censal')
    immigrants_df = immigrants_df[['Seccio_Censal', 'sumats']]
    immigrants_df = immigrants_df.sort_values(by='Seccio_Censal').reset_index(drop=True)
    

    espanyols_df = dataframe[dataframe['LLOC_NAIX_PAIS'] == 58]

    seccions_mancants = [seccio for seccio in seccions_unique if not (seccio in list(espanyols_df['Seccio_Censal']))]
    seccions_mancants_df = pd.DataFrame({'Seccio_Censal': seccions_mancants, 'Valor': [0]*len(seccions_mancants)})
    espanyols_df = pd.concat([espanyols_df, seccions_mancants_df], ignore_index=True)

    espanyols_df['sumats'] = espanyols_df.groupby('Seccio_Censal')['Valor'].transform('sum')
    espanyols_df = espanyols_df.drop_duplicates('Seccio_Censal')
    espanyols_df = espanyols_df[['Seccio_Censal', 'sumats']]
    espanyols_df = espanyols_df.sort_values(by='Seccio_Censal').reset_index(drop=True)

    total_immigrants = immigrants_df['sumats'].sum()
    total_espanyols = espanyols_df['sumats'].sum()

    quantitat_seccions = len(espanyols_df['Seccio_Censal'])

    mitjana_espanyols = total_espanyols / quantitat_seccions
    mitjana_immigrants = total_immigrants / quantitat_seccions

    #ESPANYOLS 
    diferencia_espanyols = np.array(espanyols_df['sumats']) - mitjana_espanyols

    #denominador
    espanyols_denominador = np.inner(diferencia_espanyols, diferencia_espanyols)

    #numerador
    espanyols_matriu = np.multiply(np.outer(diferencia_espanyols, diferencia_espanyols), w)
    espanyols_numerador = espanyols_matriu.sum()


    #IMMIGRANTS
    diferencia_immigrants = immigrants_df['sumats'] - mitjana_immigrants

    #denominador
    immigrants_denominador = np.inner(diferencia_immigrants, diferencia_immigrants)

    #numerador
    immigrants_matriu = np.multiply(np.outer(diferencia_immigrants, diferencia_immigrants), w)
    immigrants_numerador = immigrants_matriu.sum()

    z_i_4 = (diferencia_immigrants**4).sum()
    z_i_2 = immigrants_denominador

    W = w.sum()

    #Moran's I
    morans_espanyols = (quantitat_seccions / W )* (espanyols_numerador / espanyols_denominador)
    morans_immigrants = (quantitat_seccions / W )* (immigrants_numerador / immigrants_denominador)

    expected = -1/(quantitat_seccions - 1)
    D = quantitat_seccions*(z_i_4) / (z_i_2)**2

    S_1 = .5 * ((w + np.transpose(w))**2).sum()
    S_2 = sum((np.sum(w, axis=0) + np.sum(w,axis=1))**2)

    A = quantitat_seccions * ((quantitat_seccions**2 - 3*quantitat_seccions + 3) * S_1 - quantitat_seccions * S_2 + 3 * (W)**2)
    B = D * ((quantitat_seccions**2 - quantitat_seccions)*S_1 - 2*quantitat_seccions*S_2 + 6*(W)**2)
    C = (quantitat_seccions - 1) * (quantitat_seccions - 2) * (quantitat_seccions - 3) * (W)**2

    expected_I_2 = (A-B)/C

    variance = expected_I_2 - expected**2

    Z = (morans_immigrants - expected) / math.sqrt(variance)

    return morans_espanyols, morans_immigrants, expected, variance, Z, total_immigrants

## Plots

def plot_morans_I (data, country_name, dir):
    plt.figure(figsize=(5,3))
    plt.plot(list(data.keys()), [x[1] for x in list(data.values())[:]], marker='o', linestyle='-', color='green')
    plt.xlabel("Year")
    plt.ylabel("Index value")
    # plt.title(f"Moran's index by year in Barcelona ({country_name})")
    plt.xticks(list(data.keys())[::2], rotation=45)
    plt.grid(True, linestyle='--', alpha=0.5)
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    plt.tight_layout()
    plt.savefig(dir + f'{country_name}_year_value.png', dpi=300)
    plt.close()

def plot_morans_I_comparison (data, country_names, dir):
    colours = ['#0cb2af', '#a1c65d', '#fac723', '#f29222', '#e95e50', '#936fac']
    plt.figure(figsize=(10, 6))
    for i in range(len(data)):
        plt.plot(list(data[i].keys()), [x[1] for x in list(data[i].values())], marker='o', linestyle='-', color=colours[i], label=f'Moran\'s index ({country_names[i]})')
    plt.xlabel("Year")
    plt.ylabel("Index value")
    # plt.title(f"Moran's index by year in Barcelona ({country_names[i]})")
    plt.xticks(list(data[0].keys())[::2], rotation=45)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.legend()
    plt.tight_layout()
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    plt.savefig(dir + f'countries_comparison_year_value.png', dpi=300)
    plt.close()

In [ ]:
paisos_a_considerar_morans = [65, 108, 11, 67, 174, 58]
paisos_a_considerar_quantitat = [11, 39, 161, 167, 230, 58]
country_names_morans = ['Philippines','Italy', 'Argentina', 'France', 'United Kingdom', 'Foreigners']
country_names_quantitat = ['Argentina', 'Colombia', 'Pakistan', 'Peru', 'Venezuela', 'Foreigners']

w = calcul_pesos()

for paisos_a_considerar,country_names,dir in [[paisos_a_considerar_morans, country_names_morans, 'morans_results/higher_morans/'],[paisos_a_considerar_quantitat, country_names_quantitat,'morans_results/higher_quantity/']]:
    data = []
    for i,pais in enumerate(paisos_a_considerar):
        moransI = {}
        country_name = country_names[i]
        print(f"Working on: {country_name}")

        for filename in dataframes_naix:
            any = int(filename[5:9])
            moransI[any] = calcul_morans_1(filename, pais, w)
        
        plot_morans_I(moransI, country_name, dir)
        data.append(moransI)

    # Plot comparison
    plot_morans_I_comparison(data, country_names, dir)

    with open(dir + 'taula.csv', 'w') as f:
        f.write('Year,')
        for year in range(2002,2025):
            f.write(f'{year},')
        f.write('\n')
        for i,country in enumerate(country_names):
            f.write(country + '\n')
            index_vals = [x[1] for x in list(data[i].values())[::-1]]
            z_vals = [x[4] for x in list(data[i].values())[::-1]]
            f.write('MI value,')
            for iv in index_vals:
                f.write('{:.3f}'.format(iv) + ',')
            f.write('\n')
            f.write('MI Z score,')
            for zv in z_vals:
                f.write('{:.3f}'.format(zv) + ',')
            f.write('\n')

Working on: Philippines


C:\Users\xenia\AppData\Local\Temp\ipykernel_33696\2013037646.py:136: RuntimeWarning: overflow encountered in scalar multiply
  C = (quantitat_seccions - 1) * (quantitat_seccions - 2) * (quantitat_seccions - 3) * (W)**2
C:\Users\xenia\AppData\Local\Temp\ipykernel_33696\2013037646.py:136: RuntimeWarning: overflow encountered in scalar multiply
  C = (quantitat_seccions - 1) * (quantitat_seccions - 2) * (quantitat_seccions - 3) * (W)**2
C:\Users\xenia\AppData\Local\Temp\ipykernel_33696\2013037646.py:136: RuntimeWarning: overflow encountered in scalar multiply
  C = (quantitat_seccions - 1) * (quantitat_seccions - 2) * (quantitat_seccions - 3) * (W)**2
C:\Users\xenia\AppData\Local\Temp\ipykernel_33696\2013037646.py:136: RuntimeWarning: overflow encountered in scalar multiply
  C = (quantitat_seccions - 1) * (quantitat_seccions - 2) * (quantitat_seccions - 3) * (W)**2
C:\Users\xenia\AppData\Local\Temp\ipykernel_33696\2013037646.py:136: RuntimeWarning: overflow encountered in scalar multiply

Working on: Italy


C:\Users\xenia\AppData\Local\Temp\ipykernel_33696\2013037646.py:136: RuntimeWarning: overflow encountered in scalar multiply
  C = (quantitat_seccions - 1) * (quantitat_seccions - 2) * (quantitat_seccions - 3) * (W)**2
C:\Users\xenia\AppData\Local\Temp\ipykernel_33696\2013037646.py:136: RuntimeWarning: overflow encountered in scalar multiply
  C = (quantitat_seccions - 1) * (quantitat_seccions - 2) * (quantitat_seccions - 3) * (W)**2
C:\Users\xenia\AppData\Local\Temp\ipykernel_33696\2013037646.py:136: RuntimeWarning: overflow encountered in scalar multiply
  C = (quantitat_seccions - 1) * (quantitat_seccions - 2) * (quantitat_seccions - 3) * (W)**2
C:\Users\xenia\AppData\Local\Temp\ipykernel_33696\2013037646.py:136: RuntimeWarning: overflow encountered in scalar multiply
  C = (quantitat_seccions - 1) * (quantitat_seccions - 2) * (quantitat_seccions - 3) * (W)**2
C:\Users\xenia\AppData\Local\Temp\ipykernel_33696\2013037646.py:136: RuntimeWarning: overflow encountered in scalar multiply

Working on: Argentina


...

C:\Users\xenia\AppData\Local\Temp\ipykernel_33696\2013037646.py:136: RuntimeWarning: overflow encountered in scalar multiply
  C = (quantitat_seccions - 1) * (quantitat_seccions - 2) * (quantitat_seccions - 3) * (W)**2
C:\Users\xenia\AppData\Local\Temp\ipykernel_33696\2013037646.py:136: RuntimeWarning: overflow encountered in scalar multiply
  C = (quantitat_seccions - 1) * (quantitat_seccions - 2) * (quantitat_seccions - 3) * (W)**2
C:\Users\xenia\AppData\Local\Temp\ipykernel_33696\2013037646.py:136: RuntimeWarning: overflow encountered in scalar multiply
  C = (quantitat_seccions - 1) * (quantitat_seccions - 2) * (quantitat_seccions - 3) * (W)**2
C:\Users\xenia\AppData\Local\Temp\ipykernel_33696\2013037646.py:136: RuntimeWarning: overflow encountered in scalar multiply
  C = (quantitat_seccions - 1) * (quantitat_seccions - 2) * (quantitat_seccions - 3) * (W)**2
C:\Users\xenia\AppData\Local\Temp\ipykernel_33696\2013037646.py:136: RuntimeWarning: overflow encountered in scalar multiply

Working on: Venezuela


C:\Users\xenia\AppData\Local\Temp\ipykernel_33696\2013037646.py:136: RuntimeWarning: overflow encountered in scalar multiply
  C = (quantitat_seccions - 1) * (quantitat_seccions - 2) * (quantitat_seccions - 3) * (W)**2
C:\Users\xenia\AppData\Local\Temp\ipykernel_33696\2013037646.py:136: RuntimeWarning: overflow encountered in scalar multiply
  C = (quantitat_seccions - 1) * (quantitat_seccions - 2) * (quantitat_seccions - 3) * (W)**2
C:\Users\xenia\AppData\Local\Temp\ipykernel_33696\2013037646.py:136: RuntimeWarning: overflow encountered in scalar multiply
  C = (quantitat_seccions - 1) * (quantitat_seccions - 2) * (quantitat_seccions - 3) * (W)**2
C:\Users\xenia\AppData\Local\Temp\ipykernel_33696\2013037646.py:136: RuntimeWarning: overflow encountered in scalar multiply
  C = (quantitat_seccions - 1) * (quantitat_seccions - 2) * (quantitat_seccions - 3) * (W)**2
C:\Users\xenia\AppData\Local\Temp\ipykernel_33696\2013037646.py:136: RuntimeWarning: overflow encountered in scalar multiply

Working on: Foreigners


C:\Users\xenia\AppData\Local\Temp\ipykernel_33696\2013037646.py:136: RuntimeWarning: overflow encountered in scalar multiply
  C = (quantitat_seccions - 1) * (quantitat_seccions - 2) * (quantitat_seccions - 3) * (W)**2
C:\Users\xenia\AppData\Local\Temp\ipykernel_33696\2013037646.py:136: RuntimeWarning: overflow encountered in scalar multiply
  C = (quantitat_seccions - 1) * (quantitat_seccions - 2) * (quantitat_seccions - 3) * (W)**2
C:\Users\xenia\AppData\Local\Temp\ipykernel_33696\2013037646.py:136: RuntimeWarning: overflow encountered in scalar multiply
  C = (quantitat_seccions - 1) * (quantitat_seccions - 2) * (quantitat_seccions - 3) * (W)**2
C:\Users\xenia\AppData\Local\Temp\ipykernel_33696\2013037646.py:136: RuntimeWarning: overflow encountered in scalar multiply
  C = (quantitat_seccions - 1) * (quantitat_seccions - 2) * (quantitat_seccions - 3) * (W)**2
C:\Users\xenia\AppData\Local\Temp\ipykernel_33696\2013037646.py:136: RuntimeWarning: overflow encountered in scalar multiply

# Mapes

In [8]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# Funcions

## Càlculs

def contar_immigrants_per_seccio (dataframe_name, pais):

    dataframe =  pd.read_csv(dataframe_name)
    dataframe['Valor'] = pd.to_numeric(dataframe['Valor'].replace('..', 2))

    seccions_unique = set(dataframe['Seccio_Censal'])

    if pais == 58:
        immigrants_df = dataframe[dataframe['LLOC_NAIX_PAIS'] != pais]
    else:
        immigrants_df = dataframe[dataframe['LLOC_NAIX_PAIS'] == pais]

    seccions_mancants = [seccio for seccio in seccions_unique if not (seccio in list(immigrants_df['Seccio_Censal']))]
    seccions_mancants_df = pd.DataFrame({'Seccio_Censal': seccions_mancants, 'Valor': [0]*len(seccions_mancants)})
    immigrants_df = pd.concat([immigrants_df, seccions_mancants_df], ignore_index=True)

    immigrants_df['sumats'] = immigrants_df.groupby('Seccio_Censal')['Valor'].transform('sum')
    immigrants_df = immigrants_df.drop_duplicates('Seccio_Censal')
    immigrants_df = immigrants_df[['Seccio_Censal', 'sumats']]
    immigrants_df = immigrants_df.sort_values(by='Seccio_Censal').reset_index(drop=True)

    return immigrants_df['sumats'].tolist()

## Plots

def plot_dades_mapa_seccio_censal (data, country_name, dir):
    gdf = gpd.read_file("data/seccio-censal.geojson") 

    gdf['ordre_seccio'] = gdf["DISTRICTE"] + gdf["SEC_CENS"]
    gdf = gdf.sort_values(by='ordre_seccio').reset_index(drop=True)
    gdf['data'] = data

    fig, ax = plt.subplots(figsize=(10, 10))
    gdf.plot(
        ax=ax, 
        column='data', 
        edgecolor='black', 
        legend=True, 
        legend_kwds={'shrink': 0.6, 'label': '', 'orientation': 'vertical'}, 
        missing_kwds={'color': 'white'}, 
        cmap='Greens'
    )
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_xticks([])
    ax.set_yticks([])

    for spine in ax.spines.values():
        spine.set_visible(False)

    plt.savefig(dir + f"{country_name}_mapa_barcelona_total_immigrants_2024.png", dpi=300)
    plt.close()

In [9]:
paisos_a_considerar_morans = [65, 108, 11, 67, 174, 58]
paisos_a_considerar_quantitat = [11, 39, 161, 167, 230, 58]
country_names_morans = ['Philippines','Italy', 'Argentina', 'France', 'United Kingdom', 'Foreigners']
country_names_quantitat = ['Argentina', 'Colombia', 'Pakistan', 'Peru', 'Venezuela', 'Foreigners']
dataframe_name = 'data/2024_pad_mdbas_lloc-naix-pais_lloc-naix-continent_sexe.csv'

for paisos_a_considerar,country_names,dir in [[paisos_a_considerar_morans, country_names_morans, 'morans_results/higher_morans/'],[paisos_a_considerar_quantitat, country_names_quantitat,'morans_results/higher_quantity/']]:
    for i,pais in enumerate(paisos_a_considerar):
        country_name = country_names[i]
        print(f'Working on: {country_name}')
        immigrants_per_seccio = contar_immigrants_per_seccio(dataframe_name, pais)
        plot_dades_mapa_seccio_censal(immigrants_per_seccio, country_name, dir)


Working on: Philippines
Working on: Italy
Working on: Argentina
Working on: France
Working on: United Kingdom
Working on: Foreigners
Working on: Argentina
Working on: Colombia
Working on: Pakistan
Working on: Peru
Working on: Venezuela
Working on: Foreigners


# Exemples

In [4]:
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

# Funcions

## Processament

def calcul_pesos():
    gdf = gpd.read_file("data/seccio-censal.geojson")

    neighbours_named = {}

    for index, area in gdf.iterrows():
        name = index
        touching_names = gdf[gdf.geometry.touches(area.geometry)].index.tolist()
        neighbours_named[name] = touching_names
        
    w = [[0]*len(gdf) for _ in range (len(gdf))]

    for i,neighbours in neighbours_named.items():
        for j in neighbours:
            w[i][j] = 1

    return np.array(w)

## Càlculs

def calcul_morans_1_exemples(data, w):
    total = sum(data)

    quantitat_seccions = len(data)

    mitjana = total / quantitat_seccions

    diferencia = data - mitjana

    denominador = np.inner(diferencia, diferencia)

    matriu = np.multiply(np.outer(diferencia, diferencia), w)
    numerador = matriu.sum()

    W = w.sum()

    #Moran's I
    morans_I = (quantitat_seccions / W )* (numerador / denominador)

    return morans_I

## Plots

def plot_exemple_mapa_seccio_censal (data, valor, example_name):
    gdf = gpd.read_file("data/seccio-censal.geojson") 

    gdf['ordre_seccio'] = gdf["DISTRICTE"] + gdf["SEC_CENS"]
    gdf = gdf.sort_values(by='ordre_seccio').reset_index(drop=True)
    gdf['data'] = data

    fig, ax = plt.subplots(figsize=(10, 10))
    gdf.plot(
        ax=ax, 
        column='data', 
        edgecolor='black', 
        legend=True, 
        legend_kwds={'shrink': 0.6, 'label': '', 'orientation': 'vertical'}, 
        missing_kwds={'color': 'white'}, 
        cmap='Greens'
    )

    # Remove axis labels and ticks
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_xticks([])
    ax.set_yticks([])

    # Remove axis spines (borders)
    for spine in ax.spines.values():
        spine.set_visible(False)

    plt.savefig(f"morans_results/examples/{example_name}_" + '{:.3f}'.format(valor) + '.png', dpi=300)
    plt.close()

In [5]:
w = calcul_pesos()

# Exemple random amb soroll

rand_vals = 1 + .05*np.random.rand(1068)

moransI_rand = calcul_morans_1_exemples(rand_vals, w)

plot_exemple_mapa_seccio_censal(rand_vals, moransI_rand, "random_noise")

# Exemple taulell escacs

chess_vals = []
for i in range(1068):
    chess_vals.append(i%2)
chess_vals = np.array(chess_vals)

moransI_chess = calcul_morans_1_exemples(chess_vals, w)
plot_exemple_mapa_seccio_censal(chess_vals, moransI_chess, "chessboard")

# Exemple grup conjunt

grouped_vals = []
for i in range(1068):
    if i < 300:
        grouped_vals.append(1)
    else:
        grouped_vals.append(0)
grouped_vals = np.array(grouped_vals)

moransI_grouped = calcul_morans_1_exemples(grouped_vals, w)
plot_exemple_mapa_seccio_censal(grouped_vals, moransI_grouped, "grouped")